In [181]:
import requests
from bs4 import BeautifulSoup

# - FYIs (still need fixing): 

### name change for diff case IDs (older include UKSC)
### name change for older citation #s (Neutral Citation v Neutral Citation Number v Neutral Citation Number(s))

### FUNCTION todos: 

define each function & tweak to iterate over many pages (list of each dict output)

concatenate each dictionary into single dataframe (two dataframes? one for more detail and one for all details)

write tests??? 

add urls to dictionary

# MAIN??

In [172]:
def fix_date(date):
    months = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 
              'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12',
              'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 
              'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    if len(date) <= 17:
        return(f'{months[date[3:-5]]}/{date[:2]}/{date[-4:]}')
    else: 
        return(f'{months[date[3:6]]}/{date[:2]}/{date[7:11]}')

In [150]:
strips = list(soup.stripped_strings)
if 'Judgment date' in strips:
    datedex = strips.index('Judgment date') + 1
date = strips[datedex]

print(fix_date(date))

02/09/2011


In [180]:
def landing_to_dict(case):
    url = f'https://www.supremecourt.uk/cases/{case}.html'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')

    strips = list(soup.stripped_strings)
    datedex = strips.index('Judgment date') + 1
    date = strips[datedex]
    new_date = fix_date(date)

    if case != 'uksc-2013-0124' and case != 'uksc-2010-0195' and case != 'uksc-2009-0016' \
    and case != 'uksc-2009-0018' and case != 'uksc-2009-0015' and case != 'uksc-2013-0273' \
    and case != 'uksc-2009-0210':
        if new_date >= '04/29/2019': 
            case_dict = func1(soup)
        elif new_date >= '03/01/2019': 
            case_dict = func2(soup)
        elif new_date >= '10/29/2019':
            case_dict = func3(soup)
    elif case == 'uksc-2013-0124' or case == 'uksc-2010-0195':
        case_dict = func2(soup)
    elif case == 'uksc-2009-0016' or case == 'uksc-2009-0018' or case == 'uksc-2009-0015':
        case_dict = func4(soup)
    elif case == 'uksc-2013-0273':
        case_dict = func5(soup)
    elif case == 'uksc-2009-0210':
        case_dict = func6(soup)
    
    case_dict['URL'] = url
    return case_dict

# - 29 April 2020 - present (func1)

this works for ALL 2020(april)-2022 cases

In [133]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2020-0066.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [99]:
def func1(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'): 
        case_name = detail.find('span').string
        case_id = detail.find('h3', class_='sc-access').string
        case_id = case_id.replace('Case ID: ', '')
        strips = list(soup.stripped_strings)
        if 'Issue' in strips:
            issues = strips.index('Issue')
        else: 
            issues = strips.index('Issue(s)')
        facts = strips.index('Facts')
        judgment = strips.index('Judgment appealed')
        issues_string = ''.join(strips[issues+1:facts])
        facts_string = ''.join(strips[facts+1:judgment])
        keys = []
        for d in detail.find_all('h4'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Keys': keys [-8:], 'Values': values[-8:]})
        case_details.append({'Keys': keys, 'Values': values})
        case_details = case_details[0]
    
    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Keys']))}
    match_dict = {}
    match_dict['Judgment date'] = case_dict['Judgment date']
    match_dict['Neutral citation number'] = case_dict['Neutral citation']
    match_dict['Case ID'] = case_id
    match_dict['Justices'] = case_dict['Justices']
    match_dict['Name'] = case_name
    return match_dict

{'Judgment date': '12 January 2022',
 'Neutral citation number': '[2022] UKSC 1',
 'Case ID': '2020/0066',
 'Justices': 'Lord Briggs, Lord Sales, Lord Leggatt, Lord Burrows, Lady Rose',
 'Name': 'FirstPort Property Services Ltd (Appellant) v Settlers Court RTM Company and others (Respondents)'}

In [ ]:
def func1_alt(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'): 
        case_name = detail.find('span').string
        case_id = detail.find('h3', class_='sc-access').string
        case_id = case_id.replace('Case ID: ', '')
        strips = list(soup.stripped_strings)
        if 'Issue' in strips:
            issues = strips.index('Issue')
        else: 
            issues = strips.index("Issue(s)")
        facts = strips.index('Facts')
        judgment = strips.index('Judgment appealed')
        issues_string = ''.join(strips[issues+1:facts])
        facts_string = ''.join(strips[facts+1:judgment])
        keys = []
        for d in detail.find_all('h4'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Keys': keys [-8:], 'Values': values[-8:]})
        case_details.append({'Keys': keys, 'Values': values})
        case_details = case_details[0]
    
    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Keys']))}
    case_dict['Name'] = case_name
    case_dict['Case ID'] = case_id
    case_dict['Issues'] = issues_string
    case_dict['Facts'] = facts_string
    return case_dict

# - 01 March 2019 - 01 April 2020 (func2)

this works for ALL cases 13 March 2019 - April 1 2020


WORKS FOR "R v Chaytor and others (Appellants)" 01 DEC 2010
**uksc-2010-0195**

WORKS FOR "Greater Glasgow Health Board..." 17 DEC 2014
**uksc-2013-0124**

In [139]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2013-0124.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [53]:
def func2(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'):
        case_name = detail.find('span').string
        keys = []
        for d in detail.find_all('h3', class_='sc-access'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Keys': keys, 'Values': values})
        case_details = case_details[0]

    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
    case_dict['Name'] = case_name
    return case_dict

{'Judgment date': '17 Dec 2014',
 'Neutral citation number': '[2014] UKSC 68',
 'Case ID': 'UKSC 2013/0124',
 'Justices': 'Lady Hale, Lord Wilson, Lord Reed, Lord Hughes, Lord Hodge',
 'Name': 'Greater Glasgow Health Board (Appellant) v Doogan and another (Respondents) (Scotland)'}

# - 29 October 2009 - 06 March 2019 (func3)

works from October 2009 (first case) - 6 March 2019

In [144]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0180.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [145]:
def func3(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'):
        case_name = detail.find('span').string
        keys = []
        for d in detail.find_all('h5'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
        case_details = case_details[0]

    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
    case_dict['Name'] = case_details['Name']
    return case_dict

{'Judgment date': '09 Feb 2011 & 03 Nov 2010',
 'Neutral citation number': '[2011] UKSC 6 & [2010] UKSC 45',
 'Case ID': 'UKSC 2009/0180',
 'Justices': 'Lord Phillips, Lord Hope, Lord Rodger, Lord Walker, Lady Hale, Lord Brown, Lord Mance, Lord Neuberger, Lord Collins',
 'Name': 'Manchester City Council (Respondent) v Pinnock (Appellant)'}

# - January 2010 (func4)

cases with MORE THAN two citation numbers (first three Jan 2010) 

**uksc-2009-0016**

**uksc-2009-0018**

**uksc-2009-0015**

In [59]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0016.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [60]:
def func4(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'):
        case_name = detail.find('span').string
        keys = []
        for d in detail.find_all('h5'):
            key = d.string
            keys.append(key)
        keys.remove('Neutral citation number(s)')
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        citations = []
        for d in detail.find('ul'):
            cit = d.string
            citations.append(cit)
        citations = [citation for citation in citations if citation != '\n']
        citations = ' & '.join(citations)
        case_details.append({'Keys': keys, 'Values': values})
        case_details = case_details[0]

    case_dict = {test_details['Keys'][i]: test_details['Values'][i] for i in range(3)}
    case_dict['Neutral citation number(s)'] = citations
    case_dict['Name'] = case_name
    return case_dict

{'Judgment date': '27 Jan 2010',
 'Case ID': 'UKSC 2009/0016',
 'Justices': 'Lord Phillips, Lord Hope, Lord Rodger, Lord Walker, Lady Hale, Lord Brown, Lord Mance',
 'Neutral citation number(s)': '[2010] UKSC 1 & [2010] UKSC 2 & [2010] UKSC 5',
 'Name': "Her Majesty's Treasury (Respondents) v Mohammed Jabar Ahmed and others (FC) (Appellants) "}

# - 03 December 2014 (func5)

Loveridge (Appellant) **uksc-2013-0273**

In [61]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2013-0273.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [62]:
def func5(soup)
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'):
        case_name = detail.find('span').string
        keys = ['Judgment date']
        for d in detail.find_all('h4'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
        case_details = case_details[0]

    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Values']))}
    case_dict['Name'] = case_details['Name']
    return case_dict

{'Judgment date': '03 Dec 2014',
 'Neutral citation number': '[2014] UKSC 65',
 'Case ID': 'UKSC 2013/0273',
 'Justices': 'Lord Neuberger, Lord Wilson, Lord Sumption, Lord Carnwath, Lord Toulson',
 'Name': 'Loveridge (Appellant) v Mayor and Burgesses of the London Borough of Lambeth (Respondent) '}

# - 01 December 2010 (func6)

Spiller and another (Appellant) **uksc-2009-0210**

In [63]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0210.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [64]:
def func6(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'):
        case_name = detail.find('h3').string
        keys = []
        for d in detail.find_all('h5'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
        case_details = case_details[0]

    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
    case_dict['Name'] = case_details['Name']
    return case_dict

{'Judgment date': '01 Dec 2010',
 'Neutral citation number': '[2010] UKSC 53',
 'Case ID': 'UKSC 2009/0210',
 'Justices': 'Lord Phillips, Lord Rodger, Lord Walker, Lord Brown, Sir John Dyson',
 'Name': 'Spiller and another (Appellants) v Joseph and others (Respondents)'}

# Test - scraping multipar summaries

In [83]:
url = "https://www.supremecourt.uk/cases/uksc-2020-0195.html"
html = requests.get(url).content
soup = BeautifulSoup(html,'html.parser')

strips = list(soup.stripped_strings)
if "Issue" in strips:
    issues = strips.index("Issue")
else: 
    issues = strips.index("Issue(s)")
facts = strips.index('Facts')
judgment = strips.index('Judgment appealed')

issues_string = ''.join(strips[issues+1:facts])
facts_string = ''.join(strips[facts+1:judgment])

print(facts_string)

Mr Maduro was re-elected President of Venezuela in May 2018. Mr Guaido is the President of the National Assembly of Venezuela. Mr Guaido claims that the May 2018 election was flawed and that he is Interim President of Venezuela.Both parties have appointed different Boards to the Central Bank of Venezuela. These Boards have issued conflicting instructions concerning nearly US$1 billon of Venezuela’s international reserves, held in the Bank of England’s vaults, and approximately US$120 million held by court-appointed receivers on behalf of the Central Bank of Venezuela, following a payment from Deutsche Bank under a swap contract.The High Court ordered a trial to determine whether the Guaido Board or the Maduro Board had control over the disputed sums. The High Court found for the Guaido Board. The Maduro Board successfully appealed to the Court of Appeal. The Guaido Board now appeals to the Supreme Court.


In [76]:
print(list(soup.stripped_strings))

['Maduro Board of the Central Bank of Venezuela (Respondent/Cross-Appellant) v Guaidó Board of the Central Bank of Venezuela (Appellant/Cross-Respondent) - The Supreme Court', 'Skip to Primary Navigation', 'Skip to Content', 'Go to the home page', 'Go to Latest news', 'Go to the sitemap page', 'Go to search', 'Go to the Decided cases', 'Go to the Current cases', 'Go to the Complaints Policies and Judicial Conduct', 'Go to the terms and conditions page', 'Go to the contact us page', 'Go to the accessibility statement page', 'Skip to main content', 'Home', 'Accessibility', 'Education', 'Press office', 'Contact us', 'FAQs', 'Careers', 'Cymraeg', 'Bookmark', 'Case details', 'Home', 'Current cases', 'Decided cases', 'Court procedures', 'Visiting The Court', 'About The Supreme Court', 'Latest', 'news', 'Current cases', 'Maduro Board of the Central Bank of Venezuela (Respondent/Cross-Appellant) v Guaidó Board" of the Central Bank of Venezuela (Appellant/Cross-Respondent)', 'Case ID: 2020/0195

# Test: multiple functions combined

In [183]:
def func2(soup):
    case_details = []
    for detail in soup.find_all('div', class_='panel-filled'):
        case_name = detail.find('span').string
        keys = []
        for d in detail.find_all('h3', class_='sc-access'):
            key = d.string
            keys.append(key)
        values = []
        for d in detail.find_all('p'):
            value = d.string
            values.append(value)
        case_details.append({'Keys': keys, 'Values': values})
        case_details = case_details[0]

    case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
    case_dict['Name'] = case_name
    return case_dict

def fix_date(date):
    months = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 
              'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12',
              'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 
              'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    if len(date) <= 17:
        return(f'{months[date[3:-5]]}/{date[:2]}/{date[-4:]}')
    else: 
        return(f'{months[date[3:6]]}/{date[:2]}/{date[7:11]}')
    
def landing_to_dict(case):
    url = f'https://www.supremecourt.uk/cases/{case}.html'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')

    strips = list(soup.stripped_strings)
    datedex = strips.index('Judgment date') + 1
    date = strips[datedex]
    new_date = fix_date(date)

    if case != 'uksc-2013-0124' and case != 'uksc-2010-0195' and case != 'uksc-2009-0016' \
    and case != 'uksc-2009-0018' and case != 'uksc-2009-0015' and case != 'uksc-2013-0273' \
    and case != 'uksc-2009-0210':
        if new_date >= '04/29/2019': 
            case_dict = func1(soup)
        elif new_date >= '03/01/2019': 
            case_dict = func2(soup)
        elif new_date >= '10/29/2019':
            case_dict = func3(soup)
    elif case == 'uksc-2013-0124' or case == 'uksc-2010-0195':
        case_dict = func2(soup)
    elif case == 'uksc-2009-0016' or case == 'uksc-2009-0018' or case == 'uksc-2009-0015':
        case_dict = func4(soup)
    elif case == 'uksc-2013-0273':
        case_dict = func5(soup)
    elif case == 'uksc-2009-0210':
        case_dict = func6(soup)
    
    case_dict['URL'] = url
    return case_dict

print(landing_to_dict('uksc-2013-0124'))

{'Judgment date': '17 Dec 2014', 'Neutral citation number': '[2014] UKSC 68', 'Case ID': 'UKSC 2013/0124', 'Justices': 'Lady Hale, Lord Wilson, Lord Reed, Lord Hughes, Lord Hodge', 'Name': 'Greater Glasgow Health Board (Appellant) v Doogan and another (Respondents) (Scotland)', 'URL': 'https://www.supremecourt.uk/cases/uksc-2013-0124.html'}
